# IPython interface for running GPU DSM + fitting on cluster

In [0]:
from dsm_tools import ssh_init, job_start, job_done, job_transfer_clean, uuid, os, server, username, password, Calculation, CalculationStar, CalculationLinear, IterationChain
from fdt_fit import fdt_fit, fdt, fdtvec
from gt_fit import Gt_MMM_vec, Gp_MMM, Gdp_MMM, Gp_MMM_vec, Gdp_MMM_vec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [0]:
matplotlib.style.use('default')

In [0]:
def mix_spectra(w, x, y):
    xWeight = w #Fraction of linear
    yWeight = 1 - w #Fraction of star

    #Calculate p_eq of the mix
    mix_lambdaArr = np.append(x.lambdaArr, y.lambdaArr)
    mix_gArr = np.append(np.multiply(x.gArr, x.lambdaArr)/np.dot(x.gArr, x.lambdaArr) * xWeight, np.multiply(y.gArr, y.lambdaArr)/np.dot(y.gArr, y.lambdaArr) * yWeight)

    #Sort by relaxation times tau
    sort = np.argsort(mix_lambdaArr)
    mix_lambdaArr = mix_lambdaArr[sort]
    mix_gArr = mix_gArr[sort]

    return mix_lambdaArr, np.divide(mix_gArr,mix_lambdaArr)/np.sum(np.divide(mix_gArr,mix_lambdaArr))

In [0]:
nChains = 2000
CDtoggle = 0
simTime = 1000000
num_gpu = 8

#Linear chain
nk = 50

#Star chain
nArms = 3
nkArms = [7, 7, 7]

In [0]:
#Create simulations
x, y = CalculationLinear('x', nk, 0, nChains, num_gpu, simTime), CalculationStar('y', nArms, nkArms, 0, nChains, num_gpu, simTime)

In [0]:
#Load simulations
x, y = CalculationLinear('x'), CalculationStar('y')

In [0]:
weights = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0] #weight fraction of linear chains in star-linear blend
iterations = map(lambda x: IterationChain(x),weights)

#Create iteration chains for different blends
iterations[0].add((CalculationLinear('linear_'+str(iterations[0].w)+'_'+str(len(iterations[0].chain)+1), nk, 1, nChains, num_gpu, simTime)))
[i.add((CalculationLinear('linear_'+str(i.w)+'_'+str(len(i.chain)+1), nk, 1, nChains, num_gpu, simTime), CalculationStar('star_'+str(i.w)+'_'+str(len(i.chain)+1), nArms, nkArms, 1, nChains, num_gpu, simTime))) for i in iterations[1:-1]]
iterations[-1].add((CalculationStar('star_'+str(iterations[-1].w)+'_'+str(len(iterations[-1].chain)+1), nArms, nkArms, 1, nChains, num_gpu, simTime)))

To start a claculation of a **linear chain**, call `CalculationLinear(Nc, CDtoggle, Nchains, Ngpus, simTime)`

To start a claculation of a **star-branched chain**, call `CalculationStar(nArms, NcArms, CDtoggle, Nchains, Ngpus, simTime)`

### Zeroth iteration (pure linear, pure star; no constraint dynamics)

In [0]:
x.calc()
y.calc()

In [0]:
x.check_self()
y.check_self()

In [0]:
x.token

In [0]:
x.plot_fdt_results()
x.plot_gt_results()

In [0]:
y.plot_fdt_results()
y.plot_gt_results()

## Longest relaxation time for a pure linear and a pure star

In [0]:
#Calculate p_cr for different blends
iterations[0].chain[-1].set_pcd_cr_input(x.lambdaArr,x.gArr)

for i in iterations[1:-1]:
    a,b = mix_spectra(i.w, x, y)
    i.chain[-1][0].set_pcd_cr_input(a,b)
    i.chain[-1][1].set_pcd_cr_input(a,b)

iterations[-1].chain[-1].set_pcd_cr_input(y.lambdaArr,y.gArr)

In [0]:
#pure linear
iterations[0].chain[-1].calc()

In [0]:
iterations[0].chain[-1].check_self()

In [0]:
print 'Pure linear'
iterations[0].chain[-1].plot_fdt_results()
iterations[0].chain[-1].plot_gt_results()

In [0]:
#pure star
iterations[-1].chain[-1].calc()

In [0]:
iterations[-1].chain[-1].check_self()

In [0]:
print 'Pure star'
iterations[-1].chain[-1].plot_fdt_results()
iterations[-1].chain[-1].plot_gt_results()

In [0]:
iterations[1].chain[-1][0].calc()
iterations[1].chain[-1][1].calc()

In [0]:
iterations[1].chain[-1][0].check_self()
iterations[1].chain[-1][1].check_self()

In [0]:
print 'Linear (90% linear + 10% star)'
iterations[1].chain[-1][0].plot_fdt_results()
iterations[1].chain[-1][0].plot_gt_results()

In [0]:
print 'Star (90% linear + 10% star)'
iterations[1].chain[-1][1].plot_fdt_results()
iterations[1].chain[-1][1].plot_gt_results()

In [0]:
iterations[2].chain[-1][0].calc()
iterations[2].chain[-1][1].calc()

In [0]:
iterations[2].chain[-1][0].check_self()
iterations[2].chain[-1][1].check_self()

In [0]:
print 'Linear (80% linear + 20% star)'
iterations[2].chain[-1][0].plot_fdt_results()
iterations[2].chain[-1][0].plot_gt_results()

In [0]:
print 'Star (80% linear + 20% star)'
iterations[2].chain[-1][1].plot_fdt_results()
iterations[2].chain[-1][1].plot_gt_results()

In [0]:
iterations[3].chain[-1][0].calc()
iterations[3].chain[-1][1].calc()

In [0]:
iterations[3].chain[-1][0].check_self()
iterations[3].chain[-1][1].check_self()

In [0]:
print 'Linear (70% linear + 30% star)'
iterations[3].chain[-1][0].plot_fdt_results()
iterations[3].chain[-1][0].plot_gt_results()

In [0]:
print 'Star (70% linear + 30% star)'
iterations[3].chain[-1][1].plot_fdt_results()
iterations[3].chain[-1][1].plot_gt_results()

In [0]:
iterations[4].chain[-1][0].calc()
iterations[4].chain[-1][1].calc()

In [0]:
iterations[4].chain[-1][0].check_self()
iterations[4].chain[-1][1].check_self()

In [0]:
print 'Linear (60% linear + 40% star)'
iterations[4].chain[-1][0].plot_fdt_results()
iterations[4].chain[-1][0].plot_gt_results()

In [0]:
print 'Star (60% linear + 40% star)'
iterations[4].chain[-1][1].plot_fdt_results()
iterations[4].chain[-1][1].plot_gt_results()

In [0]:
iterations[5].chain[-1][0].calc()
iterations[5].chain[-1][1].calc()

In [0]:
iterations[5].chain[-1][0].check_self()
iterations[5].chain[-1][1].check_self()

In [0]:
print 'Linear (50% linear + 50% star)'
iterations[5].chain[-1][0].plot_fdt_results()
iterations[5].chain[-1][0].plot_gt_results()

In [0]:
print 'Star (50% linear + 50% star)'
iterations[5].chain[-1][1].plot_fdt_results()
iterations[5].chain[-1][1].plot_gt_results()

In [0]:
iterations[6].chain[-1][0].calc()
iterations[6].chain[-1][1].calc()

In [0]:
iterations[6].chain[-1][0].check_self()
iterations[6].chain[-1][1].check_self()

In [0]:
print 'Linear (40% linear + 60% star)'
iterations[6].chain[-1][0].plot_fdt_results()
iterations[6].chain[-1][0].plot_gt_results()

In [0]:
print 'Star (40% linear + 60% star)'
iterations[6].chain[-1][1].plot_fdt_results()
iterations[6].chain[-1][1].plot_gt_results()

In [0]:
iterations[7].chain[-1][0].calc()
iterations[7].chain[-1][1].calc()

In [0]:
iterations[7].chain[-1][0].check_self()
iterations[7].chain[-1][1].check_self()

In [0]:
print 'Linear (30% linear + 70% star)'
iterations[7].chain[-1][0].plot_fdt_results()
iterations[7].chain[-1][0].plot_gt_results()

In [0]:
print 'Star (30% linear + 70% star)'
iterations[7].chain[-1][1].plot_fdt_results()
iterations[7].chain[-1][1].plot_gt_results()

In [0]:
iterations[8].chain[-1][0].calc()
iterations[8].chain[-1][1].calc()

In [0]:
iterations[8].chain[-1][0].check_self()
iterations[8].chain[-1][1].check_self()

In [0]:
iterations[8].chain[-1][0].plot_fdt_results()
iterations[8].chain[-1][0].plot_gt_results()

In [0]:
iterations[8].chain[-1][1].plot_fdt_results()
iterations[8].chain[-1][1].plot_gt_results()

In [0]:
iterations[9].chain[-1][0].calc()
iterations[9].chain[-1][1].calc()

In [0]:
iterations[9].chain[-1][0].check_self()
iterations[9].chain[-1][1].check_self()

In [0]:
iterations[9].chain[-1][0].plot_fdt_results()
iterations[9].chain[-1][0].plot_gt_results()

In [0]:
iterations[9].chain[-1][1].plot_fdt_results()
iterations[9].chain[-1][1].plot_gt_results()

### Look at the longest relaxation time of star-branched

In [0]:
iterations[1].chain[-1][1].lambdaArr

In [0]:
iterations[2].chain[-1][1].lambdaArr

In [0]:
iterations[3].chain[-1][1].lambdaArr

In [0]:
iterations[4].chain[-1][1].lambdaArr

In [0]:
iterations[5].chain[-1][1].lambdaArr

In [0]:
iterations[6].chain[-1][1].lambdaArr

In [0]:
iterations[7].chain[-1][1].lambdaArr

In [0]:
iterations[8].chain[-1][1].lambdaArr

In [0]:
iterations[9].chain[-1][1].lambdaArr

In [0]:
iterations[10].chain[-1].lambdaArr

In [0]:
fig = plt.figure(figsize=(12, 10))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution for star")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$\mathbb{P}(\tau)$')

ax1.scatter(iterations[1].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[1].chain[-1][1].lambdaArr, iterations[1].chain[-1][1].gArr))/np.dot(iterations[1].chain[-1][1].lambdaArr, iterations[1].chain[-1][1].gArr), label=r'90% linear')
ax1.scatter(iterations[2].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[2].chain[-1][1].lambdaArr, iterations[2].chain[-1][1].gArr))/np.dot(iterations[2].chain[-1][1].lambdaArr, iterations[2].chain[-1][1].gArr), label=r'80% linear')
ax1.scatter(iterations[3].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[3].chain[-1][1].lambdaArr, iterations[3].chain[-1][1].gArr))/np.dot(iterations[3].chain[-1][1].lambdaArr, iterations[3].chain[-1][1].gArr), label=r'70% linear')
ax1.scatter(iterations[4].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[4].chain[-1][1].lambdaArr, iterations[4].chain[-1][1].gArr))/np.dot(iterations[4].chain[-1][1].lambdaArr, iterations[4].chain[-1][1].gArr), label=r'60% linear')
ax1.scatter(iterations[5].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[5].chain[-1][1].lambdaArr, iterations[5].chain[-1][1].gArr))/np.dot(iterations[5].chain[-1][1].lambdaArr, iterations[5].chain[-1][1].gArr), label=r'50% linear')
ax1.scatter(iterations[6].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[6].chain[-1][1].lambdaArr, iterations[6].chain[-1][1].gArr))/np.dot(iterations[6].chain[-1][1].lambdaArr, iterations[6].chain[-1][1].gArr), label=r'40% linear')
ax1.scatter(iterations[7].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[7].chain[-1][1].lambdaArr, iterations[7].chain[-1][1].gArr))/np.dot(iterations[7].chain[-1][1].lambdaArr, iterations[7].chain[-1][1].gArr), label=r'30% linear')
ax1.scatter(iterations[8].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[8].chain[-1][1].lambdaArr, iterations[8].chain[-1][1].gArr))/np.dot(iterations[8].chain[-1][1].lambdaArr, iterations[8].chain[-1][1].gArr), label=r'20% linear')
ax1.scatter(iterations[9].chain[-1][1].lambdaArr, np.cumsum(np.multiply(iterations[9].chain[-1][1].lambdaArr, iterations[9].chain[-1][1].gArr))/np.dot(iterations[9].chain[-1][1].lambdaArr, iterations[9].chain[-1][1].gArr), label=r'10% linear')
ax1.scatter(iterations[10].chain[-1].lambdaArr, np.cumsum(np.multiply(iterations[10].chain[-1].lambdaArr, iterations[10].chain[-1].gArr))/np.dot(iterations[10].chain[-1].lambdaArr, iterations[10].chain[-1].gArr), label=r'0% linear')
leg = ax1.legend()
ax1.set_xscale('log')
#ax1.set_yscale('log')

plt.show()

## Longest relaxation time in blend

In [0]:
w = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0]

In [0]:
taud_fdt = [max(iterations[0].chain[0].lambdaArr), max([max(i.lambdaArr) for i in iterations[1].chain[0]]), max([max(i.lambdaArr) for i in iterations[2].chain[0]]), max([max(i.lambdaArr) for i in iterations[3].chain[0]]), max([max(i.lambdaArr) for i in iterations[4].chain[0]]), max([max(i.lambdaArr) for i in iterations[5].chain[0]]), max([max(i.lambdaArr) for i in iterations[6].chain[0]]), max([max(i.lambdaArr) for i in iterations[7].chain[0]]), max([max(i.lambdaArr) for i in iterations[8].chain[0]]), max([max(i.lambdaArr) for i in iterations[9].chain[0]]), max(iterations[-1].chain[0].lambdaArr)]

In [0]:
taud_gt = [max(iterations[0].chain[0].gt_lambdaArr), max([max(i.gt_lambdaArr) for i in iterations[1].chain[0]]), max([max(i.gt_lambdaArr) for i in iterations[2].chain[0]]), max([max(i.gt_lambdaArr) for i in iterations[3].chain[0]]), max([max(i.gt_lambdaArr) for i in iterations[4].chain[0]]), max(iterations[-1].chain[0].gt_lambdaArr)]

In [0]:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)

fig = plt.figure(figsize=(10, 8))

ax1 = fig.add_subplot(111)

ax1.set_title("Longest relaxation time in star/linear blend")
ax1.set_xlabel(r'$\phi_L$ - fraction of linear chains')
ax1.set_ylabel(r'$\tau_d/\tau_c$')

ax1.scatter(w, taud_fdt)
#plt.text(-0.1, 00, r'Pure stars', fontsize=20)

ax1.set_yscale('log')

plt.show()

In [0]:
max(iterations[0].chain[0].gt_lambdaArr)

### Look at the longest relaxation time of linear

In [0]:
iterations[0].chain[-1].lambdaArr

In [0]:
iterations[1].chain[-1][0].lambdaArr

In [0]:
iterations[2].chain[-1][0].lambdaArr

In [0]:
iterations[3].chain[-1][0].lambdaArr

In [0]:
iterations[4].chain[-1][0].lambdaArr

In [0]:
iterations[5].chain[-1][0].lambdaArr

In [0]:
iterations[6].chain[-1][0].lambdaArr

In [0]:
iterations[7].chain[-1][0].lambdaArr

In [0]:
iterations[8].chain[-1][0].lambdaArr

In [0]:
iterations[9].chain[-1][0].lambdaArr

# Convergence of pure linear

In [0]:
iterations[0].add((CalculationLinear('linear_'+str(iterations[0].w)+'_'+str(len(iterations[0].chain)+1), nk, 1, nChains, num_gpu, simTime)))

In [0]:
iterations[0].chain[-1].set_pcd_cr_input(iterations[0].chain[-2].lambdaArr,iterations[0].chain[-2].gArr)

In [0]:
iterations[0].chain[-1].calc()

In [0]:
iterations[0].chain[-1].check_self()

In [0]:
x.lambdaArr

In [0]:
iterations[0].chain[0].token

In [0]:
iterations[0].chain[0].lambdaArr

In [0]:
iterations[0].chain[-1].lambdaArr

In [0]:
x.fdt_result_x

In [0]:
x.plot_fit_results()

In [0]:
iterations[0].chain[-1].plot_fit_results()

In [0]:
fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution for linear")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$\mathbb{P}(\tau)$')

ax1.scatter(x.lambdaArr, np.cumsum(np.multiply(x.lambdaArr, x.gArr))/np.dot(x.lambdaArr, x.gArr), label=r'Iteration 0')
ax1.scatter(iterations[0].chain[0].lambdaArr, np.cumsum(np.multiply(iterations[0].chain[0].lambdaArr, iterations[0].chain[0].gArr)/np.dot(iterations[0].chain[0].lambdaArr, iterations[0].chain[0].gArr)), label=r'Iteration 1')
ax1.scatter(iterations[0].chain[1].lambdaArr, np.cumsum(np.multiply(iterations[0].chain[1].lambdaArr, iterations[0].chain[1].gArr)/np.dot(iterations[0].chain[1].lambdaArr, iterations[0].chain[1].gArr)), label=r'Iteration 2')
# ax1.scatter(iterations[0].chain[2].lambdaArr, np.cumsum(np.multiply(iterations[0].chain[2].lambdaArr, iterations[0].chain[2].gArr)/np.dot(iterations[0].chain[2].lambdaArr, iterations[0].chain[2].gArr)), label=r'Iteration 3')
leg = ax1.legend()
ax1.set_xscale('log')
#ax1.set_yscale('log')

plt.show()

### Compare $f_d(t)$

In [0]:
x.fdt_result_y

In [0]:
fig = plt.figure(figsize=(12, 8))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$f_d(t)$')

ax1.scatter(x.fdt_result_x, x.fdt_result_y, label=r'Iteration 0')
ax1.plot(x.fdt_result_x, fdtvec(time=x.fdt_result_x, params=np.append(x.lambdaArr, x.gArr)), c='k')
ax1.scatter(iterations[0].chain[0].fdt_result_x, iterations[0].chain[0].fdt_result_y, label=r'Iteration 1')
ax1.plot(iterations[0].chain[0].fdt_result_x, fdtvec(time=iterations[0].chain[0].fdt_result_x, params=np.append(iterations[0].chain[0].lambdaArr, iterations[0].chain[0].gArr)), c='k')
ax1.scatter(iterations[0].chain[1].fdt_result_x, iterations[0].chain[1].fdt_result_y, label=r'Iteration 2')
ax1.plot(iterations[0].chain[1].fdt_result_x, fdtvec(time=iterations[0].chain[1].fdt_result_x, params=np.append(iterations[0].chain[1].lambdaArr, iterations[0].chain[1].gArr)), c='k')
# ax1.scatter(iterations[0].chain[2].fdt_result_x, iterations[0].chain[2].fdt_result_y, label=r'Iteration 3')
# ax1.plot(iterations[0].chain[2].fdt_result_x, fdtvec(time=iterations[0].chain[2].fdt_result_x, params=np.append(iterations[0].chain[2].lambdaArr, iterations[0].chain[2].gArr)), c='k')
leg = ax1.legend()
plt.xlim(xmin=0.1)
# plt.ylim(ymax=1.0)
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

In [0]:
f = open('fdt_lin_0.dat','w')
for i in zip(x.fdt_result_x, x.fdt_result_y):
    f.write(str(i[0])+'\t'+str(i[1])+'\n')
f.close()

In [0]:
f = open('fdt_lin_1.dat','w')
for i in zip(iterations[0].chain[0].fdt_result_x, iterations[0].chain[0].fdt_result_y):
    f.write(str(i[0])+'\t'+str(i[1])+'\n')
f.close()

In [0]:
f = open('fdt_lin_2.dat','w')
for i in zip(iterations[0].chain[1].fdt_result_x, iterations[0].chain[1].fdt_result_y):
    f.write(str(i[0])+'\t'+str(i[1])+'\n')
f.close()

### Compare $G(t)$

In [0]:
omegaPoints = 1000
omegaMin = -8
omegaMax = 5
omegaArr=10**(omegaMin+(np.array(range(omegaPoints), float) + 1.0)/omegaPoints*(omegaMax-omegaMin))

fig3 = plt.figure(figsize=(16, 6))

ax1 = fig3.add_subplot(121)

ax1.set_title("Relaxation modulus G(t)")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'log residuals')

ax1.scatter(x.gt_result_x, x.gt_result_y, label=r'Iteration 0')
ax1.plot(x.gt_result_x, sum(x.gt_gArr)*Gt_MMM_vec(time=x.gt_result_x, params=np.append(x.gt_lambdaArr, x.gt_gArr)), c='k')
ax1.scatter(iterations[0].chain[0].gt_result_x, iterations[0].chain[0].gt_result_y, c='r', label=r'Iteration 1')
ax1.plot(iterations[0].chain[0].gt_result_x, sum(iterations[0].chain[0].gt_gArr)*Gt_MMM_vec(time=iterations[0].chain[0].gt_result_x, params=np.append(iterations[0].chain[0].gt_lambdaArr, iterations[0].chain[0].gt_gArr)), c='k')
ax1.scatter(iterations[0].chain[1].gt_result_x, iterations[0].chain[1].gt_result_y, label=r'Iteration 2')
ax1.plot(iterations[0].chain[1].gt_result_x, sum(iterations[0].chain[1].gt_gArr)*Gt_MMM_vec(time=iterations[0].chain[1].gt_result_x, params=np.append(iterations[0].chain[1].gt_lambdaArr, iterations[0].chain[1].gt_gArr)), c='k')
# ax1.scatter(iterations[0].chain[2].gt_result_x, iterations[0].chain[2].gt_result_y, label=r'Iteration 3')
# ax1.plot(iterations[0].chain[2].gt_result_x, sum(iterations[0].chain[2].gt_gArr)*Gt_MMM_vec(time=iterations[0].chain[2].gt_result_x, params=np.append(iterations[0].chain[2].gt_lambdaArr, iterations[0].chain[2].gt_gArr)), c='k')
# ax1.scatter(gt_BSW_x, gt_BSW_y, label=r'Previous code with BSW fit')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = fig3.add_subplot(122)

ax2.set_title(r'Dynamic modulus $G*(\omega)$')
ax2.set_xlabel(r'$\omega$')
ax2.set_ylabel(r'$G*$')

ax2.plot(omegaArr,Gp_MMM_vec(omega=omegaArr,params=np.append(x.gt_lambdaArr, x.gt_gArr)), c='k', label=r'$G^\prime$')
ax2.plot(omegaArr,Gdp_MMM_vec(omega=omegaArr,params=np.append(x.gt_lambdaArr, x.gt_gArr)), c='k', label=r'$G^{\prime\prime}$')
ax2.plot(omegaArr,Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[0].chain[0].gt_lambdaArr, iterations[0].chain[0].gt_gArr)), c='r', label=r'$G^\prime$')
ax2.plot(omegaArr,Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[0].chain[0].gt_lambdaArr, iterations[0].chain[0].gt_gArr)), c='r', label=r'$G^{\prime\prime}$')
ax2.plot(omegaArr,Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[0].chain[1].gt_lambdaArr, iterations[0].chain[1].gt_gArr)), c='b', label=r'$G^\prime$')
ax2.plot(omegaArr,Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[0].chain[1].gt_lambdaArr, iterations[0].chain[1].gt_gArr)), c='b', label=r'$G^{\prime\prime}$')

leg = ax2.legend()
ax2.set_yscale('log')
ax2.set_xscale('log')


plt.show()

In [0]:
f = open('gt_lin_0.dat','w')
for i in zip(x.gt_result_x, x.gt_result_y):
    f.write(str(i[0])+'\t'+str(i[1])+'\n')
f.close()

In [0]:
f = open('gt_lin_1.dat','w')
for i in zip(iterations[0].chain[0].gt_result_x, iterations[0].chain[0].gt_result_y):
    f.write(str(i[0])+'\t'+str(i[1])+'\n')
f.close()

In [0]:
f = open('gt_lin_2.dat','w')
for i in zip(iterations[0].chain[1].gt_result_x, iterations[0].chain[1].gt_result_y):
    f.write(str(i[0])+'\t'+str(i[1])+'\n')
f.close()

# Convergence of pure star

In [0]:
y.lambdaArr

In [0]:
iterations[-1].chain[-1].lambdaArr

In [0]:
iterations[-1].chain[-1].token

In [0]:
fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$f_d(t)$')

ax1.scatter(y.fdt_result_x, y.fdt_result_y, label=r'Iteration 0')
ax1.plot(y.fdt_result_x, fdtvec(time=y.fdt_result_x, params=np.append(y.lambdaArr, y.gArr)), c='k')
ax1.scatter(iterations[-1].chain[0].fdt_result_x, iterations[-1].chain[0].fdt_result_y, label=r'Iteration 1')
ax1.plot(iterations[-1].chain[0].fdt_result_x, fdtvec(time=iterations[-1].chain[0].fdt_result_x, params=np.append(iterations[-1].chain[0].lambdaArr, iterations[-1].chain[0].gArr)), c='k')
# ax1.scatter(iterations[0].chain[1].fdt_result_x, iterations[0].chain[1].fdt_result_y, label=r'Iteration 2')
# ax1.plot(iterations[0].chain[1].fdt_result_x, fdtvec(time=iterations[0].chain[1].fdt_result_x, params=np.append(iterations[0].chain[1].lambdaArr, iterations[0].chain[1].gArr)), c='k')
# ax1.scatter(iterations[0].chain[2].fdt_result_x, iterations[0].chain[2].fdt_result_y, label=r'Iteration 3')
# ax1.plot(iterations[0].chain[2].fdt_result_x, fdtvec(time=iterations[0].chain[2].fdt_result_x, params=np.append(iterations[0].chain[2].lambdaArr, iterations[0].chain[2].gArr)), c='k')
leg = ax1.legend()
# plt.xlim(xmin=0.5*min(self.lambdaArr), xmax=2*max(self.lambdaArr))
plt.ylim(ymin=1e-11, ymax=2)
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

In [0]:
x.lambdaArr

## Dynamic modulus of the mixture

In [0]:
x.gt_result_y[0]

In [0]:
iterations[1].chain[0][0]

In [0]:
omegaPoints = 1000
omegaMin = -9
omegaMax = 1
omegaArr=10**(omegaMin+(np.array(range(omegaPoints), float) + 1.0)/omegaPoints*(omegaMax-omegaMin))

fig3 = plt.figure(figsize=(14, 8))

ax2 = fig3.add_subplot(111)

ax2.set_title(r'Dynamic modulus $G*(\omega)$')
ax2.set_xlabel(r'$\omega$')
ax2.set_ylabel(r'$G*$')

ax2.plot(omegaArr,iterations[0].chain[0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[0].chain[0].gt_lambdaArr, iterations[0].chain[0].gt_gArr)), c='C0', label=r'100% linear')
ax2.plot(omegaArr,iterations[0].chain[0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[0].chain[0].gt_lambdaArr, iterations[0].chain[0].gt_gArr)), c='C0')

ax2.plot(omegaArr,iterations[1].w*iterations[1].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[1].chain[0][0].gt_lambdaArr, iterations[1].chain[0][0].gt_gArr))+(1-iterations[1].w)*iterations[1].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[1].chain[0][1].gt_lambdaArr, iterations[1].chain[0][1].gt_gArr)), c='C1', label=r'90% linear')
ax2.plot(omegaArr,iterations[1].w*iterations[1].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[1].chain[0][0].gt_lambdaArr, iterations[1].chain[0][0].gt_gArr))+(1-iterations[1].w)*iterations[1].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[1].chain[0][1].gt_lambdaArr, iterations[1].chain[0][1].gt_gArr)), c='C1')

ax2.plot(omegaArr,iterations[2].w*iterations[2].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[2].chain[0][0].gt_lambdaArr, iterations[2].chain[0][0].gt_gArr))+(1-iterations[2].w)*iterations[2].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[2].chain[0][1].gt_lambdaArr, iterations[2].chain[0][1].gt_gArr)), c='C2', label=r'80% linear')
ax2.plot(omegaArr,iterations[2].w*iterations[2].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[2].chain[0][0].gt_lambdaArr, iterations[2].chain[0][0].gt_gArr))+(1-iterations[2].w)*iterations[2].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[2].chain[0][1].gt_lambdaArr, iterations[2].chain[0][1].gt_gArr)), c='C2')

ax2.plot(omegaArr,iterations[3].w*iterations[3].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[3].chain[0][0].gt_lambdaArr, iterations[3].chain[0][0].gt_gArr))+(1-iterations[3].w)*iterations[3].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[3].chain[0][1].gt_lambdaArr, iterations[3].chain[0][1].gt_gArr)), c='C3', label=r'70% linear')
ax2.plot(omegaArr,iterations[3].w*iterations[3].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[3].chain[0][0].gt_lambdaArr, iterations[3].chain[0][0].gt_gArr))+(1-iterations[3].w)*iterations[3].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[3].chain[0][1].gt_lambdaArr, iterations[3].chain[0][1].gt_gArr)), c='C3')

ax2.plot(omegaArr,iterations[4].w*iterations[4].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[4].chain[0][0].gt_lambdaArr, iterations[4].chain[0][0].gt_gArr))+(1-iterations[4].w)*iterations[4].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[4].chain[0][1].gt_lambdaArr, iterations[4].chain[0][1].gt_gArr)), c='C4', label=r'60% linear')
ax2.plot(omegaArr,iterations[4].w*iterations[4].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[4].chain[0][0].gt_lambdaArr, iterations[4].chain[0][0].gt_gArr))+(1-iterations[4].w)*iterations[4].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[4].chain[0][1].gt_lambdaArr, iterations[4].chain[0][1].gt_gArr)), c='C5')

ax2.plot(omegaArr,iterations[5].w*iterations[5].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[5].chain[0][0].gt_lambdaArr, iterations[5].chain[0][0].gt_gArr))+(1-iterations[5].w)*iterations[5].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[5].chain[0][1].gt_lambdaArr, iterations[5].chain[0][1].gt_gArr)), c='C6', label=r'50% linear')
ax2.plot(omegaArr,iterations[5].w*iterations[5].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[5].chain[0][0].gt_lambdaArr, iterations[5].chain[0][0].gt_gArr))+(1-iterations[5].w)*iterations[5].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[5].chain[0][1].gt_lambdaArr, iterations[5].chain[0][1].gt_gArr)), c='C6')

ax2.plot(omegaArr,iterations[6].w*iterations[6].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[6].chain[0][0].gt_lambdaArr, iterations[6].chain[0][0].gt_gArr))+(1-iterations[6].w)*iterations[6].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[6].chain[0][1].gt_lambdaArr, iterations[6].chain[0][1].gt_gArr)), c='C7', label=r'40% linear')
ax2.plot(omegaArr,iterations[6].w*iterations[6].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[6].chain[0][0].gt_lambdaArr, iterations[6].chain[0][0].gt_gArr))+(1-iterations[6].w)*iterations[6].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[6].chain[0][1].gt_lambdaArr, iterations[6].chain[0][1].gt_gArr)), c='C7')

ax2.plot(omegaArr,iterations[7].w*iterations[7].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[7].chain[0][0].gt_lambdaArr, iterations[7].chain[0][0].gt_gArr))+(1-iterations[7].w)*iterations[7].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[7].chain[0][1].gt_lambdaArr, iterations[7].chain[0][1].gt_gArr)), c='C8', label=r'30% linear')
ax2.plot(omegaArr,iterations[7].w*iterations[7].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[7].chain[0][0].gt_lambdaArr, iterations[7].chain[0][0].gt_gArr))+(1-iterations[7].w)*iterations[7].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[7].chain[0][1].gt_lambdaArr, iterations[7].chain[0][1].gt_gArr)), c='C8')

ax2.plot(omegaArr,iterations[8].w*iterations[8].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[8].chain[0][0].gt_lambdaArr, iterations[8].chain[0][0].gt_gArr))+(1-iterations[8].w)*iterations[8].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[8].chain[0][1].gt_lambdaArr, iterations[8].chain[0][1].gt_gArr)), c='C9', label=r'20% linear')
ax2.plot(omegaArr,iterations[8].w*iterations[8].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[8].chain[0][0].gt_lambdaArr, iterations[8].chain[0][0].gt_gArr))+(1-iterations[8].w)*iterations[8].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[8].chain[0][1].gt_lambdaArr, iterations[8].chain[0][1].gt_gArr)), c='C9')

ax2.plot(omegaArr,iterations[9].w*iterations[9].chain[0][0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[9].chain[0][0].gt_lambdaArr, iterations[9].chain[0][0].gt_gArr))+(1-iterations[9].w)*iterations[9].chain[0][1].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[9].chain[0][1].gt_lambdaArr, iterations[9].chain[0][1].gt_gArr)), c='C0', label=r'10% linear')
ax2.plot(omegaArr,iterations[9].w*iterations[9].chain[0][0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[9].chain[0][0].gt_lambdaArr, iterations[9].chain[0][0].gt_gArr))+(1-iterations[9].w)*iterations[9].chain[0][1].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[9].chain[0][1].gt_lambdaArr, iterations[9].chain[0][1].gt_gArr)), c='C0')

ax2.plot(omegaArr,iterations[10].chain[0].gt_result_y[0]*Gp_MMM_vec(omega=omegaArr,params=np.append(iterations[10].chain[0].gt_lambdaArr, iterations[10].chain[0].gt_gArr)), c='k', label=r'0% linear')
ax2.plot(omegaArr,iterations[10].chain[0].gt_result_y[0]*Gdp_MMM_vec(omega=omegaArr,params=np.append(iterations[10].chain[0].gt_lambdaArr, iterations[10].chain[0].gt_gArr)), c='k')

plt.ylim(ymin=1e-7, ymax=100)

leg = ax2.legend()
ax2.set_yscale('log')
ax2.set_xscale('log')


plt.show()

In [0]:
0.01*(340)**3.48

In [0]:
2.5*np.exp(0.6*38)

In [0]:
0.01*(340)**3.48/(2.5*np.exp(0.6*38))

In [0]:
0.01*(6)**3.48/(2.5*np.exp(0.6*15))